# Emoji expression

In [5]:
import numpy as np
import cv2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
# for data augmentation 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data preprocessing, augmentation

In [6]:
train_dir = 'data/train'
val_dir = 'data/test'

# apply data augmentation
# scaling factor -> 0-255 to 0-1 pixel values in array
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48), # image size
        batch_size=64,
        color_mode="grayscale", 
        class_mode='categorical' # type of label array return
)

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## CNN

In [7]:
emotion_model = Sequential()
# cnn 
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# attached ann layer (flat)
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
# seven emotions in datasets
emotion_model.add(Dense(7, activation='softmax'))

## Load weights (If trained)

In [8]:
emotion_model.load_weights("model.h5")

## Training
### Skip if weights already available

In [ ]:
emotion_model.compile(
    loss='categorical_crossentropy',
#     optimizer="Adam",
    optimizer=Adam(lr=0.0001, decay=1e-6),
    metrics=['accuracy']
)

# fit_generator -> when dataset is large and cant fit in memory,
# or applying data augmentation
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64, # 28709 training images
        epochs=2,
        validation_data=validation_generator,
        validation_steps=7178 // 64) # 7178 test/val images

Epoch 1/2
448/448 [==============================] - 276s 615ms/step - loss: 0.3153 - accuracy: 0.8865 - val_loss: 1.3150 - val_accuracy: 0.6221
Epoch 2/2
409/448 [==========================>...] - ETA: 22s - loss: 0.3031 - accuracy: 0.8935

In [14]:
emotion_model.save_weights('model.h5')

In [15]:
emotion_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 128)        

## Video capture: OpenCV

In [16]:
# cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # pretrianed cascade classifier
    bounding_box = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml') 
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detect faces in each frame
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        # draw rectangle
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        # predict emotion
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

array([[222, 118, 201, 201]], dtype=int32)

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 56.4 MB 58 kB/s  eta 0:00:01    |█████                           | 8.8 MB 784 kB/s eta 0:01:01     |█████▏                          | 9.0 MB 784 kB/s eta 0:01:01     |███████▊                        | 13.6 MB 773 kB/s eta 0:00:56     |████████▎                       | 14.5 MB 710 kB/s eta 0:00:59     |█████████▋                      | 17.0 MB 299 kB/s eta 0:02:12     |██████████▊                     | 18.9 MB 587 kB/s eta 0:01:04     |███████████████████▉            | 35.0 MB 918 kB/s eta 0:00:24     |████████████████████▉           | 36.7 MB 459 kB/s eta 0:00:43     |██████████████████████████▍     | 46.5 MB 287 kB/s eta 0:00:35     |█████████████████████████████   | 51.0 MB 184 kB/s eta 0:00:29
